In [1]:
import attr
import numpy as np
from thermo import Chemical
import warnings

from gibbs.models.ceos import PengRobinson78, SoaveRedlichKwong
from gibbs.mixture import Mixture
from gibbs.stability_analysis import stability_test
from gibbs.utilities import convert_bar_to_Pa

warnings.filterwarnings('ignore')

## Loading the thermodynamic models (cubic EoSs)

In [2]:
@attr.s(auto_attribs=True)
class NichitaPR:
    z: np.ndarray
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray
        
    @property
    def mixture(self):
        return Mixture(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor
        )

    @property
    def model(self):
        return PengRobinson78(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor
    
    
@attr.s(auto_attribs=True)
class NichitaSRK:
    z: np.ndarray
    Tc: np.ndarray
    Pc: np.ndarray
    acentric_factor: np.ndarray
    bip: np.ndarray
        
    @property
    def mixture(self):
        return Mixture(
            z=self.z, 
            Tc=self.Tc, 
            Pc=self.Pc, 
            acentric_factor=self.acentric_factor
        )

    @property
    def model(self):
        return SoaveRedlichKwong(
            mixture=self.mixture,
            bip=self.bip
        )

    @property
    def number_of_components(self):
        return len(self.z)

    def fugacity(self, P, T, z):
        Z_factor = self.calculate_Z(P, T, z)
        return self.model.calculate_fugacity(P, T, z, Z_factor)

    def calculate_Z(self, P, T, z):
        Z_factor = self.model.calculate_Z_minimal_energy(P, T, z)
        return Z_factor

## Case 1 (Problem 5)

In [3]:
methane = Chemical('methane')
ethane = Chemical('ethane')
nitrogen = Chemical('nitrogen')

* Feed composition 1:

In [4]:
z = np.array([0.10, 0.60, 0.30])
omegas = np.array([methane.omega, ethane.omega, nitrogen.omega])
Tcs = np.array([methane.Tc, ethane.Tc, nitrogen.Tc])
Pcs = np.array([methane.Pc, ethane.Pc, nitrogen.Pc])
kijs = np.array([
    [0.000, 0.021, 0.038],
    [0.021, 0.000, 0.080],
    [0.038, 0.080, 0.000]
])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [5]:
P = 7.6e6
T = 270

result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.06792845, 0.79846824, 0.1336033 ]))

* Feed composition 2:

In [6]:
z = np.array([0.30, 0.55, 0.15])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [7]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.24567754, 0.65703817, 0.09728428]))

* Feed composition 3:

In [8]:
z = np.array([0.38, 0.54, 0.08])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [9]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.38000792, 0.53998839, 0.08000369]))

* Feed composition 4:

In [10]:
z = np.array([0.05, 0.90, 0.05])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [11]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.04999998, 0.90000013, 0.04999989]))

## Case 2: Yarborough 8 mixture

There is something wrong or missing in this case!

In [12]:
propane = Chemical('propane')
pentane = Chemical('n-pentane')
heptane = Chemical('heptane')
decane = Chemical('n-decane')

In [13]:
z = np.array([0.8097, 0.0566, 0.0306, 0.0457, 0.0330, 0.0244])
omegas = np.array([
    methane.omega, 
    ethane.omega, 
    propane.omega,
    pentane.omega,
    heptane.omega,
    decane.omega
])
Tcs = np.array([
    methane.Tc, 
    ethane.Tc, 
    propane.Tc,
    pentane.Tc,
    heptane.Tc,
    decane.Tc
])
Pcs = np.array([
    methane.Pc, 
    ethane.Pc, 
    propane.Pc,
    pentane.Pc,
    heptane.Pc,
    decane.Pc
])
kijs = np.zeros((len(Pcs), len(Pcs)))
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

* Condition 1:

In [14]:
P = convert_bar_to_Pa(210)  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.80969155, 0.05660063, 0.03060073, 0.04570221, 0.03300237,
        0.02440251]))

* Condition 2:

In [15]:
P = convert_bar_to_Pa(210.66)  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.80969525, 0.05660019, 0.03060041, 0.04570129, 0.03300139,
        0.02440147]))

* Condition 3:

In [16]:
P = convert_bar_to_Pa(210.67)  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.80969595, 0.05660023, 0.03060035, 0.04570115, 0.03300116,
        0.02440116]))

* Condition 4:

In [17]:
P = convert_bar_to_Pa(215)  # in Pa
T = 293.78

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.80969877, 0.0566    , 0.03060022, 0.04570029, 0.03300036,
        0.02440038]))

## Case 3 (Problem 1)

In [18]:
methane = Chemical('methane')
hydrogen_sulfide = Chemical('H2S')

* Feed composition 1:

In [19]:
z = np.array([0.5, 0.5])
omegas = np.array([methane.omega, hydrogen_sulfide.omega])
Tcs = np.array([methane.Tc, hydrogen_sulfide.Tc])
Pcs = np.array([methane.Pc, hydrogen_sulfide.Pc])
kijs = np.array([
    [0.000, 0.080],
    [0.080, 0.000]
])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [20]:
P = convert_bar_to_Pa(40.53)
T = 190

result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.92558986, 0.07441014]))

* Feed composition 2:

In [21]:
z = np.array([0.9885, 0.0115])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [22]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.98850016, 0.01149984]))

* Feed composition 3:

In [23]:
z = np.array([0.9813, 0.0187])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [24]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.11500052, 0.88499948]))

The result above must be **True**.

* Feed composition 4:

In [25]:
z = np.array([0.112, 0.888])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [26]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.11199998, 0.88800002]))

The result above must be **True**.

* Feed composition 5:

In [27]:
z = np.array([0.11, 0.89])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [28]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.91928079, 0.08071921]))

## Case 4 (Problem 2)

In [29]:
methane = Chemical('methane')
propane = Chemical('propane')

### P = 50 bar

* Feed composition 1:

In [30]:
z = np.array([0.10, 0.90])
omegas = np.array([methane.omega, propane.omega])
Tcs = np.array([methane.Tc, propane.Tc])
Pcs = np.array([methane.Pc, propane.Pc])
kijs = np.array([
    [0.000, 0.029],
    [0.029, 0.000]
])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [31]:
P = convert_bar_to_Pa(50)
T = 277.6

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.09999999, 0.90000001]))

* Feed composition 2:

In [32]:
z = np.array([0.40, 0.60])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [33]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.86561368, 0.13438632]))

* Feed composition 3:

In [34]:
z = np.array([0.60, 0.40])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [35]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.19288711, 0.80711289]))

* Feed composition 4:

In [36]:
z = np.array([0.90, 0.10])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [37]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.90000004, 0.09999996]))

### P = 100 bar

* Feed composition 1:

In [38]:
z = np.array([0.40, 0.60])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [39]:
P = convert_bar_to_Pa(100)
T = 277.6

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.39999989, 0.60000011]))

* Feed composition 2:

In [40]:
z = np.array([0.68, 0.32])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [41]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.77287172, 0.22712828]))

* Feed composition 3:

In [42]:
z = np.array([0.73, 0.27])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [43]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.64984014, 0.35015986]))

* Feed composition 4:

In [44]:
z = np.array([0.90, 0.10])
model = NichitaSRK(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [45]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.90000013, 0.09999987]))

## Case 5 (Problem 4)

In [46]:
methane = Chemical('methane')
carbon_dioxide = Chemical('CO2')

* Feed composition 1:

In [47]:
z = np.array([0.90, 0.10])
omegas = np.array([methane.omega, carbon_dioxide.omega])
Tcs = np.array([methane.Tc, carbon_dioxide.Tc])
Pcs = np.array([methane.Pc, carbon_dioxide.Pc])
kijs = np.array([
    [0.000, 0.095],
    [0.095, 0.000]
])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [48]:
P = convert_bar_to_Pa(60.8)
T = 220

result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.90000028, 0.09999972]))

* Feed composition 2:

In [49]:
z = np.array([0.80, 0.20])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [50]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.5011043, 0.4988957]))

This should be **False**!

* Feed composition 3:

In [51]:
z = np.array([0.70, 0.30])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [52]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.81527769, 0.18472231]))

* Feed composition 4:

In [53]:
z = np.array([0.57, 0.43])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [54]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(True, array([0.80904786, 0.19095214]))

The result above should be **True**!

* Feed composition 5:

In [55]:
z = np.array([0.40, 0.60])
model = NichitaPR(
    z=z,
    Tc=Tcs,
    Pc=Pcs,
    acentric_factor=omegas, 
    bip=kijs
)

In [56]:
result = stability_test(model, P, T, z)

result.phase_split, result.x

(False, array([0.39999911, 0.60000089]))